<a href="https://colab.research.google.com/github/iakshaykr/AI_ML/blob/main/TF2_Beginners_Experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [14]:
tf.debugging.set_log_device_placement(True)

In [15]:
tf.executing_eagerly()

True

In [5]:
pwd

'/content'

In [8]:
ls /content/sample_data/

anscombe.json*               california_housing_train.csv  mnist_train_small.csv
california_housing_test.csv  mnist_test.csv                README.md*


In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [10]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.15268317,  0.74618095,  0.03467325,  0.07643928, -0.16576192,
        -0.17384283, -0.57636297,  0.36837286, -0.04799867, -0.05523615]],
      dtype=float32)

In [16]:
predictions.shape

(1, 10)

In [17]:
tf.nn.softmax(predictions).numpy()

array([[0.08040146, 0.1975312 , 0.09696876, 0.10110451, 0.07935676,
        0.07871807, 0.05263349, 0.13538039, 0.08927459, 0.0886308 ]],
      dtype=float32)

In [18]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [20]:
loss_fn(y_train[:1], predictions).numpy()

2.5418825

In [21]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2978 - accuracy: 0.9147
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1403 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1061 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0858 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0731 - accuracy: 0.9771


In [23]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0734 - accuracy: 0.9781 - 679ms/epoch - 2ms/step


[0.07338006049394608, 0.9781000018119812]

In [24]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [25]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.25908571e-07, 5.43322045e-08, 4.70597952e-05, 7.88460311e-05,
        8.59403260e-12, 2.00909867e-07, 1.29493855e-12, 9.99871016e-01,
        1.70622479e-06, 1.00813497e-06],
       [2.34656653e-07, 2.75566801e-03, 9.97228682e-01, 6.29099259e-06,
        3.65872189e-14, 2.75902448e-06, 4.30059714e-07, 2.72769177e-12,
        5.86747456e-06, 2.07532819e-12],
       [2.25206295e-05, 9.96342242e-01, 2.48026568e-04, 4.92382933e-05,
        1.16778734e-04, 8.34229868e-05, 1.77756316e-04, 1.66824646e-03,
        1.29138504e-03, 3.82386844e-07],
       [9.99457419e-01, 1.23726664e-08, 4.80446877e-04, 2.52861241e-07,
        2.38179587e-06, 6.33101536e-06, 3.36359117e-05, 1.59185092e-05,
        1.97091889e-08, 3.49847323e-06],
       [4.91877727e-04, 9.33856881e-10, 6.73260583e-05, 3.44439570e-07,
        9.91927028e-01, 7.78175149e-07, 4.20125070e-06, 2.93198653e-04,
        5.05736011e-07, 7.21462956e-03]], dtype=float32)>

ADVANCED

In [26]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [27]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [29]:
x_train.shape

(60000, 28, 28, 1)

In [30]:
x_test.shape

(10000, 28, 28, 1)

In [31]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [33]:
test_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [34]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [35]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [36]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [37]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [38]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [39]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result():0.2f}, '
    f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
    f'Test Loss: {test_loss.result():0.2f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
  )

Epoch 1, Loss: 0.14, Accuracy: 95.66, Test Loss: 0.07, Test Accuracy: 97.65
Epoch 2, Loss: 0.04, Accuracy: 98.61, Test Loss: 0.06, Test Accuracy: 98.06
Epoch 3, Loss: 0.02, Accuracy: 99.27, Test Loss: 0.06, Test Accuracy: 98.25
Epoch 4, Loss: 0.01, Accuracy: 99.51, Test Loss: 0.06, Test Accuracy: 98.24
Epoch 5, Loss: 0.01, Accuracy: 99.65, Test Loss: 0.06, Test Accuracy: 98.32
